In [1]:
from annotation import Label
from dino_finder import DinoFinder
import os
import cv2
from groundingdino.util.inference import load_model, load_image, predict, annotate
import json
from tqdm import tqdm
import uuid

/home/hbdesk/auto_label_GroundDINO/venv/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/hbdesk/auto_label_GroundDINO/groundingdino/models/GroundingDINO/ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [2]:
CONFIG_PATH = "./groundingdino/config/GroundingDINO_SwinT_OGC.py"
WEIGHTS_PATH = os.path.join("weights", "groundingdino_swint_ogc.pth")
FISH_FOLDER="/media/hbdesk/UNTITLED/fish_pics_org"

In [3]:
# renames the file with uuid.


# for folder in os.listdir(FISH_FOLDER):
#     fish_folder_ = os.path.join(FISH_FOLDER, folder)
#     for fish in os.listdir(fish_folder_):
#         fish_ext = os.path.splitext(fish)[1]
#         fish_pic_uuid = f"{uuid.uuid4().hex}{fish_ext}"
#         o_fpath = os.path.join(fish_folder_, fish)
#         rname_fpath = os.path.join(f"{fish_folder_}", fish_pic_uuid)
#         os.rename(o_fpath, rname_fpath)

In [6]:
def find_from_image(image_pth, prompt, bt, tt):
    text_prompt = prompt
    bt = bt
    tt = tt
    imgsrc, img = load_image(image_pth)
    boxes, logits, phrases = predict(
        model=model, image=img, caption=text_prompt, box_threshold=bt, text_threshold=tt, device="cpu"
    )
    annotated_image = annotate(imgsrc, boxes, logits, phrases)
    # cv2.imwrite(f"results/{os.path.basename(image_pth)}.png", annotated_image)
    return boxes.tolist(), logits.tolist(), phrases

In [ ]:
model = load_model(CONFIG_PATH, WEIGHTS_PATH)
for folder in tqdm(os.listdir(FISH_FOLDER)):
    fish_folder_ = os.path.join(FISH_FOLDER, folder)
    try:
        for fish in os.listdir(fish_folder_):
            if fish.endswith((".jpeg", ".png", ".JPG", ".jpg")):
                o_fpath = os.path.join(fish_folder_, fish)
                cv_image = cv2.imread(o_fpath)
                _cls = str(os.path.basename(fish_folder_))
               
                boxes, _conf, _  = find_from_image(image_pth=o_fpath, prompt="fish", bt=0.5, tt=0.22)
                print(boxes, _conf, _cls)
                if len(boxes) > 0:
                    _label = o_fpath
                    label_z = Label(boxes, _cls, cv_image, _label).create_label()
                    json_filename = os.path.splitext(_label)[0] + ".json"
                    with open(json_filename, "w") as json_output:
                        json_output.write(json.dumps(label_z, indent=2))
                else:
                    continue
    except Exception as _ayy:
        print(f"exception: {_ayy}")
        continue
        